In [1]:
!pip install sentence-transformers -q
!pip install --upgrade gensim -q

     |████████████████████████████████| 78 kB 5.0 MB/s 
     |████████████████████████████████| 3.3 MB 30.6 MB/s 
     |████████████████████████████████| 3.3 MB 38.8 MB/s 
     |████████████████████████████████| 1.2 MB 42.7 MB/s 
     |████████████████████████████████| 61 kB 545 kB/s 
     |████████████████████████████████| 895 kB 37.2 MB/s 
     |████████████████████████████████| 596 kB 42.6 MB/s 
     |████████████████████████████████| 24.1 MB 54.6 MB/s 


In [2]:
import gensim
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
import tqdm
import nltk
import re
import multiprocessing
from sentence_transformers import SentenceTransformer

In [3]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
def preprocess_text(sentence):
    # Lowercase
    sentence = sentence.lower()
    
    # Remove all non-alphabets (punctuation, numbers, new-line characters and extra-spaces)
    sentence = re.sub(r'[^a-zA-Z]+', ' ', sentence)
    sentence = sentence.replace('\n', '')
    # sentence = re.sub('\s\s+', ' ', sentence)
    
    # Tokenize & remove stop-words
    word_list = nltk.word_tokenize(sentence)    
    stopwords_list = set(nltk.corpus.stopwords.words('english'))
    word_list = [word for word in word_list if word not in stopwords_list]
    
    # Remove very small words, length < 3, they don't contribute any useful information
    word_list = [word for word in word_list if len(word) > 3]
        
    # Stem & Lemmatize
    porter_stemmer = nltk.stem.PorterStemmer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    word_list = [porter_stemmer.stem(word) for word in word_list]
    word_list = [lemmatizer.lemmatize(word) for word in word_list]
    
    sentence = ' '.join(word_list)
    
    return sentence

### Load Data

In [6]:
train_df = pd.read_csv('train.csv')

In [7]:
for index, row in train_df.iterrows():
    if row['class'] == 1:
        train_df.at[index, 'topic'] = 'World'
    elif row['class'] == 2:
        train_df.at[index, 'topic'] = 'Sports'
    elif row['class'] == 3:
        train_df.at[index, 'topic'] = 'Business'
    else:
        train_df.at[index, 'topic'] = 'Sci/Tech'

In [8]:
# Preprocess the news description
tqdm.tqdm.pandas()
train_df['news_tokenized'] = train_df['description'].progress_apply(lambda x: preprocess_text(str(x)))

100%|██████████| 120000/120000 [02:20<00:00, 851.08it/s]


### Sentence BERT (SBERT)

In [9]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')
sbert_embeddings = embedder.encode(train_df['news_tokenized'])

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Find optimal epochs for k-means

In [10]:
performance_metrics = pd.DataFrame(columns=['feature-extraction','run#', 'epoch', 'state', 'AMI','ARI','NMI'])

In [11]:
for run in range(1, 21, 1):
    print('Run #', run)
    for epoch, state in zip(range(25, 300, 15), range(2, 42, 2)):
        k_means = KMeans(n_clusters=4, init='k-means++', max_iter=epoch, random_state=state)
        k_means.fit(sbert_embeddings)
        
        ami = metrics.adjusted_mutual_info_score(train_df['class'], k_means.labels_)
        ari = metrics.adjusted_rand_score(train_df['class'], k_means.labels_)
        nmi = metrics.normalized_mutual_info_score(train_df['class'], k_means.labels_)

        # print(run, epoch, state)
        
        performance_metrics = performance_metrics.append({'feature-extraction':'SBERT', 'run#':run, 'epoch':epoch, 'state':state, 'AMI':'{:f}'.format(ami), 'ARI':'{:f}'.format(ari), 'NMI':'{:f}'.format(nmi)}, ignore_index=True)

Run # 1
Run # 2
Run # 3
Run # 4
Run # 5
Run # 6
Run # 7
Run # 8
Run # 9
Run # 10
Run # 11
Run # 12
Run # 13
Run # 14
Run # 15
Run # 16
Run # 17
Run # 18
Run # 19
Run # 20


In [12]:
performance_metrics.to_csv('performance-sbert_kmeans.csv', index=False)

In [14]:
performance_metrics[['run#','epoch','state','AMI','ARI','NMI']] = performance_metrics[['run#','epoch','state','AMI','ARI','NMI']].apply(pd.to_numeric, errors='coerce')

In [15]:
mean_performance = performance_metrics.groupby('epoch', as_index=False)[['AMI','ARI','NMI']].mean()

In [16]:
mean_performance.to_csv('mean_performance-sbert_kmeans.csv', index=False)